In [23]:
import numpy as np
import pandas as pd
import pickle

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

SEED = 42

# Define Functions

In [28]:
def print_scores(y_true, y_pred):
    print('ROCAUC score:',roc_auc_score(y_true, y_pred))
    print('Accuracy score:',accuracy_score(y_true, y_pred))
    print('F1 score:',f1_score(y_true, y_pred))
    print('Precision score:',precision_score(y_true, y_pred))
    print('Recall:',recall_score(y_true, y_pred))
    print("\n\n")

def run_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print(model)
    print_scores(y_test, y_pred)
    return score, y_pred

def tune_model(model, param_grid, n_iter, X_train, y_train):
    grid = RandomizedSearchCV(model, param_grid, verbose=20,
        scoring='roc_auc', cv=3, n_iter=n_iter)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    return best_model

# Load data

In [17]:
df = pd.read_csv('../data/Parameters_90%stability.csv')
df = df.drop(['Unnamed: 0'], axis = 1)
X_train = pd.read_csv('../data/x_train.csv')
y_train = pd.read_csv('../data/y_train.csv')
X_test = pd.read_csv('../data/x_test.csv')
y_test = pd.read_csv('../data/y_test.csv')

# Normalize data

In [29]:
def normalize(X_train, X_test):
    scaler = StandardScaler()
#     scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test

X_train, X_test = normalize(X_train, X_test)

In [48]:
rf = RandomForestClassifier()
score, y_pred = run_model(rf, X_train, y_train, X_test, y_test)

RandomForestClassifier()
ROCAUC score: 0.6545454545454547
Accuracy score: 0.8035714285714286
F1 score: 0.45
Precision score: 0.5
Recall: 0.4090909090909091





In [49]:
# Plot the feature importances 
feat_importances = pd.Series(rf.feature_importances_, 
                             index=df.drop('Stability',axis=1).columns)
feat_importances = feat_importances.sort_values(ascending=False)
feat_importances_top15 = feat_importances.head(15)
px.bar(y=feat_importances_top15, x=feat_importances_top15.index, 
       template='ggplot2', width=600)

!pip install rulefit 

In [55]:
y_train.values.ravel()

array([1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [65]:
rulefit = RuleFit()
rulefit.fit(X_train, y_train.values.ravel(), feature_names=features)

RuleFit(tree_generator=GradientBoostingRegressor(learning_rate=0.01,
                                                 max_depth=100,
                                                 max_leaf_nodes=7,
                                                 n_estimators=545,
                                                 random_state=544,
                                                 subsample=0.5))

In [91]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

rulefit = RuleFit(tree_generator=RandomForestRegressor(n_estimators = 100))
rulefit.fit(X_train, y_train.values.ravel(), feature_names=features)

RuleFit(tree_generator=RandomForestRegressor(max_leaf_nodes=2, n_estimators=566,
                                             random_state=565))

In [92]:
rulefit_preds = rulefit.predict(X_test)
y_pred=(abs(rulefit_preds)>=0.5).astype(int)

print_scores(y_test, y_pred)

ROCAUC score: 0.6717171717171717
Accuracy score: 0.8035714285714286
F1 score: 0.47619047619047616
Precision score: 0.5
Recall: 0.45454545454545453





In [94]:
rules = rulefit.get_rules()
rules = rules.sort_values('support', ascending=False)
rules.sort_values('coef')

,rule,type,coef,support,importance
2209,Gamma_H2Ot > -0.18765784054994583 & sigma_km_s...,rule,-0.164138,0.085890,0.045991
2996,Gamma_ASNt2r <= 1.8585277795791626 & Gamma_FBA...,rule,-0.137163,0.720238,0.061570
2903,sigma_km_substrate1_SSALy <= 1.858001828193664...,rule,-0.134039,0.680723,0.062488
2519,Gamma_FBA <= -0.9268584549427032 & sigma_km_su...,rule,-0.107348,0.074534,0.028194
2805,Gamma_CITtbm > 0.9530513435602188 & Gamma_VALt...,rule,-0.105171,0.075000,0.027701
...,...,...,...,...,...
1841,sigma_km_product_CO2t > 1.4309621453285217 & G...,rule,0.120817,0.041420,0.024074
2562,Gamma_ADCS <= 0.02965996414422989 & sigma_km_s...,rule,0.141998,0.024691,0.022036
1858,Gamma_ANS > 0.4531913371756673 & sigma_km_subs...,rule,0.151402,0.135484,0.051816
2748,sigma_km_product2_PPYRDC <= 0.5975904315710068...,rule,0.225689,0.121212,0.073659


In [95]:
pd.set_option('display.max_colwidth', 400) # Adjust row width to read the entire rule
pd.options.display.float_format = '{:.2f}'.format # Round decimals to 2 decimal places
rules = rulefit.get_rules() # Get the rules
rules = rules[rules['type']!='linear'] # Eliminate the existing explanatory variables
rules = rules[rules['coef'] != 0] # eliminate the insignificant rules
rules = rules.sort_values('support', ascending=False) # Sort the rules based on "support" value
rules = rules[rules['rule'].str.len()>30] # optional: To see more complex rules, filter the long rules
rules.iloc[0:5] # Show the first 5 rules


,rule,type,coef,support,importance
2466,sigma_km_product1_AKGDbm > -1.6969951391220093 & sigma_km_substrate1_AKGDam > -1.5751768946647644 & Gamma_GLUt2r <= 1.6871368288993835 & sigma_km_product1_ALDD2y <= 1.708917498588562 & sigma_km_substrate1_GUAPRT > -1.6138514280319214,rule,-0.02,0.81,0.01
2023,Gamma_ASNt2r <= 1.9277088046073914 & sigma_km_product_ORNt3m <= 1.5606780052185059 & sigma_km_product1_DDPAm <= 1.5468677878379822 & sigma_km_substrate_RPI > -1.4137634634971619,rule,-0.07,0.80,0.03
2420,Gamma_ALAtmi > -1.6869120597839355 & sigma_km_product1_AKGMALtm <= 1.7462682723999023 & sigma_km_substrate2_ABTA > -1.620658040046692 & sigma_km_substrate7_LMPD_s_0450_c_1_256 <= 1.819287121295929,rule,-0.02,0.80,0.01
1429,sigma_km_substrate18_LMPD_s_0450_c_1_256 > -1.5359572768211365 & sigma_km_substrate2_AKGMALtm > -1.5087783336639404 & sigma_km_substrate2_NADH2_u6cm > -1.9328446984291077 & sigma_km_substrate1_ALCD2irm <= 1.8201785683631897 & Gamma_ALAtmi > -1.6869120597839355 & sigma_km_substrate1_HOMSYN2 <= 1.8170282244682312,rule,-0.00,0.78,0.00
2532,sigma_km_substrate1_IPPS > -1.5975762009620667 & sigma_km_product_ORNt3m <= 1.5908065438270569 & sigma_km_substrate_ALAt2r > -1.7072021961212158 & sigma_km_substrate2_NADS2 > -1.6770741939544678 & sigma_km_product_GCALDt <= 1.6541197896003723 & sigma_km_product2_ALCD26xi <= 1.0931100249290466,rule,-0.06,0.75,0.03
